# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('./data/song_data/')

In [5]:
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [8]:
len('SOMZWCG12A8C13C480')

18

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
# insert song record
song_datas = df.filter(items=['song_id', 'title', 'artist_id', 'year', 'duration']).values
print(song_datas)
song_data = song_datas.tolist()

[['SOMZWCG12A8C13C480' "I Didn't Mean To" 'ARD7TVE1187B99BFB1' 0 218.93179]]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
song_files = get_files('./data/song_data/')
for file in song_files:
    #print(file)
    df = pd.read_json(file, lines=True)
    song_datas = df.filter(items=['song_id', 'title', 'artist_id', 'year', 'duration']).values
    song_data = song_datas.tolist()
    for song_data in song_datas:
        try:
            cur.execute(song_table_insert, song_data)
            print(song_data)
        except Exception as e:
            print (e.pgerror)

['SOMZWCG12A8C13C480' "I Didn't Mean To" 'ARD7TVE1187B99BFB1' 0 218.93179]
['SOUDSGM12AC9618304' 'Insatiable (Instrumental Version)'
 'ARNTLGG11E2835DDB9' 0 266.39628]
['SOIAZJW12AB01853F1' 'Pink World' 'AR8ZCNI1187B9A069B' 1984 269.81832]
['SOHKNRJ12A6701D1F8' 'Drop of Rain' 'AR10USD1187B99F3F1' 0 189.57016]
['SOCIWDW12A8C13D406' 'Soul Deep' 'ARMJAGH1187FB546F3' 1969 148.03546]
['SOYMRWW12A6D4FAB14' 'The Moon And I (Ordinary Day Album Version)'
 'ARKFYS91187B98E58F' 0 267.7024]
['SOMJBYD12A6D4F8557' 'Keepin It Real (Skit)' 'ARD0S291187B9B7BF5' 0
 114.78159]
['SOQHXMF12AB0182363' 'Young Boy Blues' 'ARGSJW91187B9B1D6B' 0 218.77506]
['SONHOTT12A8C13493C' 'Something Girls' 'AR7G5I41187FB4CE6C' 1982 233.40363]
['SOFSOCN12A8C143F5D' 'Face the Ashes' 'ARXR32B1187FB57099' 2007 209.60608]
['SOXVLOJ12AB0189215' 'Amor De Cabaret' 'ARKRRTF1187B9984DA' 0 177.47546]
['SOUQQEA12A8C134B1B' 'High Tide' 'ARIG6O41187B988BDD' 0 228.5971]
['SOILPQQ12AB017E82A' 'Sohna Nee Sohna Data' 'AR1ZHYZ1187FB3C717' 0

In [11]:
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
# insert artist record
artist_datas = df.filter(items=['artist_id', 'artist_name', 'artist_latitude', 'artist_longitude','artist_location' ])
artist_datas = artist_datas.values.tolist()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
song_files = get_files('./data/song_data/')
for file in song_files:
    #print(file)
    df = pd.read_json(file, lines=True)
    artist_datas = df.filter(items=['artist_id', 'artist_name', 'artist_latitude', 'artist_longitude','artist_location' ])
    artist_datas = artist_datas.values.tolist()
    for artist_data in artist_datas:
        try:
            cur.execute(artist_table_insert, artist_data)
        except Exception as e:
            print (e.pgerror)

In [14]:
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('./data/log_data/')
log_files[1]

'/home/workspace/data/log_data/2018/11/2018-11-11-events.json'

In [16]:
filepath = log_files[1]

In [17]:
df = pd.read_json(filepath, lines=True)
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [18]:
df.shape

(95, 18)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df.loc[df['page'] == 'NextSong']
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [20]:
df.shape

(73, 18)

In [21]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [22]:
time_data = (df['ts'],
             t.dt.hour,
             t.dt.day,
             t.dt.week,
             t.dt.month,
             t.dt.year,
             t.dt.weekday
            )
column_labels = ('timestamp',
                 'hour',
                 'day',
                 'week',
                 'month',
                 'year',
                 'weekday'
            )

In [23]:
time_df = pd.DataFrame.from_dict({ col:vals for col, vals in zip(column_labels,time_data)})
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,1541903636796,2,11,45,11,2018,6
1,1541903770796,2,11,45,11,2018,6
2,1541904034796,2,11,45,11,2018,6
4,1541910973796,4,11,45,11,2018,6
5,1541911006796,4,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
user_df = df.filter(items=['userId', 'firstName', 'lastName','gender','level'])
user_df.head(3)

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (
        row.song,
        row.artist,
        row.length
    ))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        print('not found', (
            row.song,
            row.artist,
            row.length
        ))
        
    # insert songplay record
    songplay_data = (
        row.ts,
        row.userId,
        row.level,
        songid,
        artistid,
        row.sessionId,
        row.location,
        row.userAgent
    )
    try:
        cur.execute(songplay_table_insert, songplay_data)
    except Exception as e:
        print (e.pgerror)
        print(songplay_data)
    conn.commit()

not found ('Fuck Kitty', 'Frumpies', 134.47791)
not found ('By The Time This Night Is Over', 'Kenny G with Peabo Bryson', 264.75057)
not found ('God & Satan', 'Biffy Clyro', 189.83138)
not found ('Beautiful', 'HIM', 212.06159)
not found ('Supreme Balloon', 'Matmos', 1449.11628)
not found ('The One', 'Gary Allan', 259.83955)
not found ('Five Roses', 'Miracle Fortress', 200.9073)
not found ('Cuentale', 'Don Omar', 261.35465)
not found ("D'Evils", 'Jay-Z', 212.27057)
not found ('Easily (Album Version)', 'Red Hot Chili Peppers', 231.33995)
not found ('Rebels of the Sacred Heart', 'Flogging Molly', 361.9522)
not found ('Now_ Right Now', 'Reverend Horton Heat', 158.64118)
not found ('I Made A Resolution', 'Sea Wolf', 232.61995)
not found ('Lucky (Album Version)', 'Jason Mraz & Colbie Caillat', 189.6224)
not found ('Enough\x19s Enough', 'Jamie Lidell', 175.25506)
not found ('Mushaboom (Postal Service Mix)', 'Feist', 212.79302)
not found ("We're Going Out Tonight", 'Sex Slaves', 175.51628)
not

In [28]:
song_select

'\nSELECT song_id, songs.artist_id \nFROM songs JOIN artists ON artists.artist_id = songs.artist_id \nWHERE songs.title = %s\n AND  artists.name = %s\n AND  songs.duration = %s\n'

In [29]:
cur.execute(song_select, (
    'The Pretender',
    'Foo Fighters',
    271.38567
))

In [30]:
results = cur.fetchone()
results

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [31]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.